In [ ]:
from pyspark import SparkContext, SparkConf

conf=SparkConf().setMaster('local').setAppName('Assign2')
sc=SparkContext(conf=conf)


1) From the Users.xml file, find all users which are from Georgia and output to screen their DisplayName only.

In [2]:
lines=sc.textFile('Users.xml')
user_ga=lines.filter(lambda user_info: ("Georgia" or 'GA' or 'georgia' or 'ga') in user_info)
user_loc=user_ga.filter(lambda user_info: ('Location') in user_info)
user_pairs=user_loc.map(lambda info: [info.split('DisplayName=')[1].split('"')[1],
                                     info.split('Location=')[1].split('"')[1]])
user_ga_pairs=user_pairs.filter(lambda pair: ("Georgia" or 'GA' or 'georgia' or 'ga') in pair[1])
user_names=user_ga_pairs.map(lambda pair: pair[0])

user_names.collect()

['Ayush',
 'vkb',
 'Giorgi Gzirishvili',
 'Alex Azazel',
 'Dan Anton',
 'Manish Ranjan',
 'Dato Janez',
 'Neuromeda',
 'Abesalomi Gogatishvili',
 'Jenna Kwon',
 'Tedo G.',
 'mlibre',
 'Jeff',
 'Jesse Scherer',
 'Melinda Weathers',
 'N.C.W.',
 'G.T.',
 'Dato Gogolashvili',
 'David Jashi',
 'Ilia Chigogidze',
 'Tae-Sung Shin',
 'Clark Lin',
 'Gio Nari',
 'alerera',
 'Max Boonbandansook']

2) Using the Users.xml file, provide the count for all users which joined (CreationDate) in 2017. (30 points). Output this to the screen.

In [3]:
lines=sc.textFile('Users.xml')
line_date=lines.filter(lambda line: 'CreationDate' in line)
line_date_split=line_date.map(lambda line: line.split('CreationDate=')[1].split('"')[1])
line_2017=line_date_split.filter(lambda line: '2017' in line)
line_2017.count()

14239

3) Using the PostHistory file, count the number of Posts that feature the words “Spark” and “Scala”. Output this to the screen.

In [4]:
lines=sc.textFile('PostHistory.xml')
lines_Scala=lines.filter(lambda line:  'scala' in line.lower())
lines_Spark=lines.filter(lambda line:  'spark' in line.lower())
lines_both=lines_Scala.intersection(lines_Spark)
lines_both.count()

211

4) Using the PostHistory file, provide a total count of the words used by each distinct user. In other words, count all words in all posts for each user and display this to screen. You can only identify users by the UserID (30 points). You get 15 bonus points if you get the actual DisplayName of the user.

In [5]:
import re #used to clean text

In [6]:
lines=sc.textFile('PostHistory.xml')
lines_text=lines.filter(lambda line: 'Text=' in line)
lines_user=lines_text.filter(lambda line: 'UserId=' in line)
lines_pair=lines_user.map(lambda line: (int(line.split('UserId=')[1].split('"')[1]),line.split('Text=')[1].split('"')[1]))
user_text=lines_pair.map(lambda x: (x[0],re.sub('\W+', ' ',x[1]))) #remove non alpha numeric, will count contractions as two words
user_clean=user_text.map(lambda x: (x[0],
                                    re.sub('quot','',re.sub('gt','',re.sub('lt','',re.sub('xA','',re.sub('xD ', '',x[1]))))))) # removes most xml artifacts

user_cleaner=user_clean.map(lambda x: (x[0],re.sub('  ', ' ',x[1])))# remove double spaces
text_split=user_cleaner.map(lambda x: (x[0],x[1].split(' ')))#tokenize
text_len=text_split.map(lambda x: (x[0], len(x[1])-1))#ending punctuation leaves empty string at the end thus -1 to account for this
user_total=text_len.reduceByKey(lambda x,y: x+y)

#print(user_total.collect())#(UserId, Total word count from all post)

###Extra Credit
user_info=sc.textFile('Users.xml')
user_fil=user_info.filter(lambda line: 'Id=' in line)
user_fil2=user_fil.filter(lambda line: 'DisplayName=' in line)
user_id=user_fil2.map(lambda x: (int(x.split('Id=')[1].split('"')[1]),x.split('DisplayName=')[1].split('"')[1] ))

id_total_name=user_total.leftOuterJoin(user_id)
name_total=id_total_name.map(lambda x: (x[0],x[1][1],x[1][0]))
name_total.collect()#(id, username, total word count from all post)


[(180, 'Oleksi', 55),
 (375, 'nfmcclure', 2672),
 (480, 'Ger', 21),
 (555, 'GrimSqueaker', 403),
 (735, 'Kunal', 475),
 (780, 'neone4373', 2514),
 (900, 'user900', 82),
 (960, 'Abdelmawla', 214),
 (1125, 'JulianHi', 118),
 (1155, 'adesantos', 1226),
 (870, 'ArunDhaJ', 435),
 (645, 't_m', 110),
 (1350, 'magicharp', 392),
 (2790, 'MidnightLightning', 270),
 (2850, 'Ali', 93),
 (2910, 'Vikram', 57),
 (2985, 'Sylvain Peyronnet', 78),
 (525, 'Jake C.', 1340),
 (3030, 'user1913171', 138),
 (3150, 'Michael Hausenblas', 23),
 (3315, 'James', 67),
 (2475, 'Nitin Labhishetty', 141),
 (3360, 'Vinay Tiwari', 732),
 (3390, 'hnadiger', 46),
 (3510, 'David Pepper', 522),
 (3540, 'machine-wisdom', 152),
 (3615, 'user3352632', 736),
 (4620, 'Ashkan', 362),
 (4800, 'Brandon', 222),
 (4815, 'dmvianna', 121),
 (5100, 'adam', 69),
 (4710, 'Barry Carter', 214),
 (4950, 'allenwang', 131),
 (5220, 'Bas', 112),
 (5265, 'Elisa', 415),
 (5310, 'syed', 72),
 (5280, 'Tris Nefzger', 402),
 (6390, 'Prophet60091', 11

5) GRADUATE STUDENTS: Using the users.xml, comments.xml and PostHistory.xml files, produce a single file that includes the following information: DisplayName, Number of Comments, total Score and Number of posts. This file should have the users (DisplayName) sorted by score, descending from higher to lower.

In [7]:
user_info=sc.textFile('Users.xml')
user_fil=user_info.filter(lambda line: 'Id=' in line)
user_fil2=user_fil.filter(lambda line: 'DisplayName=' in line)
user_id=user_fil2.map(lambda x: (int(x.split('Id=')[1].split('"')[1]),x.split('DisplayName=')[1].split('"')[1] ))
#id, username

#user_id.take(10)

In [8]:
comments=sc.textFile('Comments.xml')
comments_fil=comments.filter(lambda line: 'Score=' in line)
comments_fil2=comments_fil.filter(lambda line: 'UserId=' in line)
id_score_one=comments_fil2.map(lambda line: (int(line.split('UserId=')[1].split('"')[1]),[int(line.split('Score=')[1].split('"')[1]),1]))
id_score_count=id_score_one.reduceByKey(lambda x,y:[x[0]+y[0],x[1]+y[1]])
##id,[score total, count]

#id_score_count.take(20)


In [9]:
posts=sc.textFile('PostHistory.xml')
posts_user=posts.filter(lambda line: 'UserId=' in line)
posts_one=posts_user.map(lambda x: (int(x.split('UserId=')[1].split('"')[1]),1))
posts_count=posts_one.reduceByKey(lambda x,y: x+y)
### id, post count

#posts_count.take(5)

e.g. (84, (('Rubens', [24, 44]), 173))# id ,name, score, comment count, post count

In [11]:
#user_id-> users.xml
#id_score_count-> comments.xml
#posts_count-> posthistory.xml

#user_comments=id_score_count.leftOuterJoin(user_id)
user_comments=user_id.join(id_score_count)
user_com_pos=user_comments.join(posts_count)
user_com_pos2=user_com_pos.map(lambda x: (x[1][0][1][0],(x[1][0][0],x[1][0][1][1],x[1][1])))#key=score, (name, comment count, post count)
score_key=user_com_pos2.sortByKey(ascending=False)# sort by score
ordered=score_key.map(lambda x: ('DisplayName='+str(x[1][0]),'NumberComments='+str(x[1][1]),'ScoreTotal='+str(x[0]),'NumberPosts='+str(x[1][2])))

#ordered.take(20)
ordered.coalesce(1).saveAsTextFile('OutputFile')